In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
data1=pd.read_csv(r'interpolation_of_ships_trial_3.csv')
data2=pd.read_csv(r'sumo results ROI.csv')

In [3]:
h=[]
for i in range(len(data2)):
    if type(data2['vds_target__boat__false_alarm_cause'][i])==str:
        h.append(i)


In [4]:
data2=data2.drop(h,axis=0).reset_index()

In [5]:
x=data1
y=data2

In [6]:
r=pd.read_pickle('23_vessels_data')

In [7]:
for i in range(len(x)):
    if (x['LENGTH'][i]==0)&(x['MMSI'][i] in r['MMSI'].unique()):
        ix=r.index[r['MMSI']==x['MMSI'][i]]
        x['LENGTH'][i]=r['Length'][ix]
        x['BREADTH'][i]=r['Breadth'][ix]
        x['NAME'][i]=r['NAME'][ix]
        

C:\ProgramData\anaconda3\envs\CNN\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\anaconda3\envs\CNN\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\anaconda3\envs\CNN\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
def haversine(lat1, lon1, lat2, lon2): 
      
    # distance between latitudes 
    # and longitudes 
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
  
    # convert to radians 
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
  
    # apply formulae 
    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
             math.cos(lat1) * math.cos(lat2)); 
    rad = 6371000
    c = 2 * math.asin(math.sqrt(a)) 
    return rad * c 

In [18]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

nei1=[]
nei2=[]
e=[]
f=[]
nei=[]
d=[]

for k in range(10):
    n1=Diff(range(len(x)),nei1)
    n2=Diff(range(len(y)),nei2)
    dis=np.array([])
    idx=[]
    o=0
    for u in n1:
        lat1 = x['LATITUDE'][u]
        lon1 = x['LONGITUDE'][u]
        a=np.array([])  
        for j in n2:
            lat2 = y['vds_target__boat__lat'][j]
            lon2 = y['vds_target__boat__lon'][j]
            a=np.append(a,haversine(lat1, lon1,lat2, lon2)) 
            idx.insert(o,(u,j))
            o=o+1
        dis=np.append(dis,a)
        
    z=[x for _,x in sorted(zip(dis,idx))]
    q=sorted(dis)
    
    for i in range(len(z)):
        if (((abs(x['LENGTH'][z[i][0]]-y['vds_target__boat__length'][z[i][1]])<30)|
            (abs(x['LENGTH'][z[i][0]]-y['vds_target__boat__length'][z[i][1]])/x['LENGTH'][z[i][0]]<0.2))&(q[i]<1000)):
            if ((z[i][0] not in e)&(z[i][1] not in f)):
                e.append(z[i][0])
                f.append(z[i][1])
                nei1.append(z[i][0])
                nei2.append(z[i][1])
                nei.append((z[i][0],z[i][1]))
                d.append(q[i])
                if k>2:
                    print('Found')
                    print(z[i][0],z[i][1])

C:\ProgramData\anaconda3\envs\CNN\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in longlong_scalars


In [14]:
integrated=pd.DataFrame()

In [15]:
for i in range(len(nei)):
    a=nei[i][0]
    b=nei[i][1]
    m=pd.DataFrame([[x['MMSI'][a],y['vds_target__boat__lat'][b],y['vds_target__boat__lon'][b],x['LATITUDE'][a],x['LONGITUDE'][a],
                    d[i],y['vds_target__boat__xpixel'][b],y['vds_target__boat__ypixel'][b],y['vds_target__boat__length'][b],
                     x['LENGTH'][a],y['vds_target__boat__width'][b],x['BREADTH'][a],y['vds_target__boat__inc_ang'][b],x['NAME'][a],x['AISTYPE'][a],
                   y['vds_target__boat__nr_pixels'][b], float(y['vds_target__boat__max_value'][b].split(',')[2]),
                   float(y['vds_target__boat__significance'][b].split(',')[2])]],
                   columns=['MMSI','Detected_lat','Detected_lon','AIS_lat','AIS_lon','Distance_error','xpixel','ypixel',
                                               'LENGTH_detected','LENGTH','BREADTH_detected','BREADTH','incidence_angle','NAME','AISTYPE','Total Pixel','Max_value','Significance'])
    
    integrated=pd.concat([integrated,m],axis=0,ignore_index=True,sort=False)
    

In [20]:
integrated.to_csv('Integrated_2_23Oct2018.csv',index=False)

In [34]:
integrated

,MMSI,Detected_lat,Detected_lon,AIS_lat,AIS_lon,Distance_error,xpixel,ypixel,LENGTH_detected,LENGTH,BREADTH_detected,BREADTH,incidence_angle,NAME,AISTYPE,Total Pixel,Max_value,Significance
0,419097600,18.931621,72.866465,18.935000,72.870100,14.214094,8901,11876,87,72,55,20,35.963,TAG-8,90,53,896.0,28.286
1,419001050,18.943907,72.869259,18.925739,72.869175,15.636808,8898,11739,93,66,45,16,35.961,TAG 19,70,45,700.0,22.150
2,419611000,18.884949,72.873521,18.903909,72.872510,15.877154,8730,12361,36,54,17,11,35.871,SEAMELODY,52,7,124.0,2.791
3,419264000,18.935131,72.860501,18.941300,72.861800,17.268777,8970,11850,79,58,47,12,36.000,ALBATROSS 07,99,40,1078.0,33.480
4,419000860,18.949235,72.868023,18.933380,72.866875,18.731173,8922,11684,56,65,41,16,35.974,"4 TAG 13 Name: NAME, dtype: object",0,26,338.0,9.931
5,419001356,18.938612,72.858113,18.946700,72.845100,21.823371,9002,11817,91,120,44,17,36.017,KEDAR NATH,70,41,571.0,17.216
6,419172000,18.949390,72.879864,18.941200,72.842500,22.415250,8800,11659,107,79,57,15,35.909,MV GURU PRASAD,72,66,502.0,16.012
7,419901486,18.909989,72.935027,18.873700,72.825000,22.833786,8147,11972,22,19,10,8,35.555,NaN,30,2,140.0,3.337
8,616443000,18.951358,72.857422,18.946510,72.844213,25.267416,9036,11682,54,46,45,9,36.036,ASHENA,37,27,415.0,12.024
9,419001069,18.959043,72.852601,18.956778,72.861743,27.050762,9102,11610,22,25,17,8,36.071,"19 SIRI Name: NAME, dtype: object",0,4,216.0,6.101


In [19]:
integrated

,MMSI,Detected_lat,Detected_lon,AIS_lat,AIS_lon,Distance_error,xpixel,ypixel,LENGTH_detected,LENGTH,BREADTH_detected,BREADTH,incidence_angle,NAME,AISTYPE,Total Pixel,Max_value,Significance
0,341611000,18.924825,72.866922,18.924800,72.866800,13.130130,8882,11947,85,110,66,14,35.953,S CAS,52,59,297.0,8.351
1,419104000,18.956207,72.859282,18.956300,72.859200,13.464897,9027,11627,96,82,43,14,36.031,NAFISA I,70,44,964.0,30.349
2,419001201,18.924717,72.864094,18.924693,72.863961,14.214094,8911,11954,99,80,63,18,35.968,WILCHIEF 1,39,63,987.0,31.710
3,419093100,18.951980,72.858420,18.951900,72.858300,15.440130,9027,11674,56,32,41,9,36.031,MV CHANDRA NEPTUNE,32,26,654.0,20.047
4,419000882,18.895360,72.843614,18.895500,72.843600,15.636808,9061,12308,191,183,87,32,36.049,OAKTREE,80,173,1425.0,49.788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,419000294,18.929555,72.857723,18.926805,72.864345,760.673251,8987,11915,68,66,41,16,36.009,SCI PAWAN,90,29,178.0,4.625
64,419001084,18.960521,72.860646,18.960300,72.868300,805.283897,9022,11578,63,34,54,10,36.028,"22 SAGAR 5 Name: NAME, dtype: object",0,37,655.0,20.716
65,419000490,18.888402,72.863001,18.894800,72.866700,810.904747,8846,12344,81,74,36,16,35.933,"7 RADHA KRISHNA IV Name: NAME, dtype: object",0,32,129.0,2.966
66,538007298,18.905034,72.876251,18.900400,72.870100,827.173189,8744,12141,170,199,89,32,35.878,INDIGO BREEZE,70,157,787.0,23.908
